In [2]:
#Excluded participants with Packer negLL less than 150.
#Get the loglikelihoods of a given data set as a measure of how easy the model
#can generate that dataset
import pickle, math
import pandas as pd
import sqlite3
import random
execfile('Imports.py')
import Modules.Funcs as funcs
from Modules.Classes import Simulation
from Modules.Classes import CopyTweak
from Modules.Classes import CopyTweakRep
from Modules.Classes import Packer
from Modules.Classes import PackerRep
from Modules.Classes import ConjugateJK13
from Modules.Classes import RepresentJK13
from scipy.stats import stats as ss
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

pearson = True
if pearson:
    corrtype = 'p'
else:
    corrtype = 's'
#Bootstrap parameters
nbootstraps = 1000

savefilename='modelvsppt{}_t.pdf'.format(corrtype)
bestparmdb = "pickles/chtc_gs_best_params_corr{}.p".format(corrtype)
modeleaseDB = "pickles/modelease_corr{}.p".format(corrtype)
#Some plotting options
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 20}
show_p = False #show pearson r in plots
show_s = False #show spearman tho in plots

#Specify simulation values
N_SAMPLES = 10000
WT_THETA = 1.5
MIN_LL = 1e-10

# Specify default dataname
dataname_def = 'midbot'#'nosofsky1986'#'NGPMG1994'
participant_def = 'all'
unique_trials_def = 'all'
dataname = dataname_def
execfile('validate_data.py')


plt.rc('font', **font)

# get data from pickle
with open(pickledir+src, "rb" ) as f:
    trials = pickle.load( f )

# get best params pickle
#bestparmdb = "pickles/chtc_gs_best_params_all_data_e1_e2.p"
#bestparmdb = "pickles/chtc_gs_best_params_corrs.p"
with open(bestparmdb, "rb" ) as f:
    best_params_t = pickle.load( f )

#Rebuild it into a smaller dict
best_params = dict()
for modelname in best_params_t.keys():    
    best_params[modelname] = dict()
    for i,parmname in enumerate(best_params_t[modelname]['parmnames']):
        parmval = best_params_t[modelname]['bestparmsll']
        best_params[modelname][parmname] = parmval[i]
modelList = [Packer,CopyTweak,ConjugateJK13,RepresentJK13]
#modelList = [CopyTweak,CopyTweakRep,Packer, RepresentJK13,]                            

#Specify plot order
modelPlotOrder = np.array([[Packer,RepresentJK13],[CopyTweak,ConjugateJK13]])
#modelPlotOrder = np.array([[CopyTweak,CopyTweakRep],[Packer,RepresentJK13]])

#Prepare matched database    
matchdb='../cat-assign/data_utilities/cmp_midbot.db'
        
unique_trials = 'all'
trials.task = task

#Get learning data
data_assign_file = '../cat-assign/data/experiment.db'
con = sqlite3.connect(data_assign_file)
info = pd.read_sql_query("SELECT * from participants", con)
assignment = pd.read_sql_query("SELECT * FROM assignment", con)
stimuli = pd.read_sql_query("SELECT * from stimuli", con).as_matrix()
con.close()
#Get generation data
data_generate_file = 'experiment-midbot.db'
con = sqlite3.connect(data_generate_file)
stats = pd.read_sql_query("SELECT * from betastats", con)
con.close()

#Get unique ppts
pptlist = []#np.array([]);
for i,row in info.iterrows():
    pptlist += [row.pptmatch]
        #    pptlist = np.concatenate((pptlist,trial['participant']))

#Specify a few more bootstrap parameters
nsamples = len(pptlist)
nsamprange = range(nsamples)


pptlist = np.unique(pptlist)
#see if ll_global exists as a pickle, otherwise construct new ll
ll_loadSuccess = {}
for model in modelList:
    ll_loadSuccess[model.model] = False
    
try:
    with open(modeleaseDB, "rb" ) as f:
        ll_global = pickle.load( f )
        #Check that model name exists in global, otherwise run it again
        for model in modelList:
            if model.model in ll_global.keys():
                ll_loadSuccess[model.model] = True
except:
    ll_global = dict()
    #ll_loadSuccess = False
    
# options for the optimization routine
options = dict(
    method = 'Nelder-Mead',
    options = dict(maxiter = 500, disp = False),
    tol = 0.01,
) 


#Prepare corr var
if 'tso' not in ll_global.keys():
    pptdata,tso = funcs.prep_corrvar(info,assignment,stimuli,stats,WT_THETA)
    ll_global['tso'] = tso
    ll_global['pptdata'] = pptdata
else:
    tso = ll_global['tso']

for model_obj in modelList:
    #model_obj = Packer
    model_name = model_obj.model
    if not ll_loadSuccess[model_name]:
        #Get log likelihoods
        ll_list = []
        print_ct = 0
        for ppt in pptlist:
            #since info contains the new mapping of ppts, and pptlist contains old,
            #convert ppt to new
            pptNew = ppt #
            pptOld = funcs.getMatch(pptNew,matchdb,fetch='Old')    
            pptloc = info['pptmatch']==pptNew
            #Get alphas with an ugly line of code
            As_num  = eval(info['stimuli'].loc[pptloc].as_matrix()[0])[0:4];
            As = stimuli[As_num,:]
            params  = best_params[model_name]
            pptcondition = info['condition'].loc[pptloc].as_matrix()[0];
            pptbeta = eval(info['stimuli'].loc[pptloc].as_matrix()[0])[4:8];
            nstim = len(pptbeta);    
            #Get weights
            ranges = stats[['xrange','yrange']].loc[stats['participant']==pptOld]
            params['wts'] = funcs.softmax(-ranges, theta = WT_THETA)[0]
            if model_obj ==  ConjugateJK13 or model_obj == RepresentJK13:
                params['wts'] = 1.0 - params['wts']
            #transform parms
            model = model_obj([As], params)
            params = model.parmxform(params, direction = 1)
            
            # Get all permutations of pptbeta and make a new trialObj for it
            nbetapermute = math.factorial(nstim)
            betapermute = [];
            raw_array = np.zeros((1,nbetapermute))
            categories = [As_num,pptbeta]
            #Get loglikelihoods
            raw_array_ll = Simulation.loglike_allperm(params, model_obj, categories, stimuli, permute_category = 1)
            ll_list += [raw_array_ll]
            #print categories,raw_array_ll
            print_ct = funcs.printProg(ppt,print_ct,steps = 1, breakline = 20, breakby = 'char')

        #Re-organise the lists
        ll_list = np.atleast_2d(ll_list)
        pptlist2d = np.atleast_2d(pptlist)
        ll = np.concatenate((pptlist2d,ll_list),axis=0).T

        #sort
        ll = ll[ll[:,1].argsort()]
        #Add third col of zeros
        ll = np.concatenate((ll,np.atleast_2d(np.zeros(len(ll))).T),axis=1)    
    
        #attach ppt errors
        for i, row in info.iterrows():
            #fh, ax = plt.subplots(1,2,figsize = (12,6))
            ppt  = row.participant
            pptAssign = assignment.loc[assignment['participant']==ppt].sort_values('trial')
            nTrials = len(pptAssign)
            accuracyEl = float(sum(pptAssign.correctcat == pptAssign.response))/nTrials


            pptNew = row.pptmatch
            #Prepare to plot configuration
            #get matched data
            #matched = funcs.getMatch(pptmatch,matchdb)
            #Add participant mean error to ll matrix
            ll[ll[:,0]==pptNew,2] = 1-accuracyEl
        
        ll_global[model_name] = ll
        
        #Save pickle for faster running next time
        with open(modeleaseDB, "wb" ) as f:
            pickle.dump(ll_global, f)
#fh,axs = plt.subplots(2,int(np.ceil(len(modelList)/2)), figsize=(20,8))
fh,axs = plt.subplots(2,int(np.ceil(len(modelList)/2)), figsize=(15,15))
plt.tight_layout(w_pad=1,h_pad=5.0,rect=(.05,.05,.95,.95))
#Inclusion list (pptmatch)
include = [26., 121., 112.,  95.,  32.,  89.,  85.,  38.,   6., 114.,  98.,
         5.,  72.,  94.,  17.,  99.,  54.,  80.,  51., 117.,  18.,  88.,
        40.,  15.,  25.,  66.,  41., 111., 115.,  67.,  75.,  53.,  44.]

for m,model_obj in enumerate(modelList):
    exclude = []
    model_loc = modelPlotOrder==model_obj
    ax = axs[model_loc][0]
    model_name = model_obj.model
    model_print = model_obj.modelprint
    llt = ll_global[model_name].copy()
    #include only specific individuals
    ll  = []
    for row in llt:
        if row[0] in include:
            ll += [row]
        else:
            exclude += [row[0]]
    ll = np.array(ll)
    nsamples = ll.shape[0]
    nsamprange = range(nsamples)


    #Get correlations
    corr_p = ss.pearsonr(ll[:,1],ll[:,2])
    corr_s = ss.spearmanr(ll[:,1],ll[:,2])
    cov = np.cov(ll[:,1],ll[:,2])
    #Run bootstrap
    corr_bs_p = []
    corr_bs_s = []
    bestfitlines = []
    if not pearson:
        rankorderx_base = np.array(range(len(ll[:,1])))
        rankordery_base = ll[:,2].argsort()
    for bsi in range(nbootstraps):
        bs_samples = [random.choice(nsamprange) for ct in nsamprange]
        ll_bs = ll[bs_samples,:]
        corr_bs_p += [ss.pearsonr(ll_bs[:,1],ll_bs[:,2])[0]]            
        corr_bs_s += [ss.spearmanr(ll_bs[:,1],ll_bs[:,2])[0]]
        if pearson:
            #Find best fit line        
            bscoeff = np.polyfit(ll_bs[:,1],ll_bs[:,2],1)
            xrange_bs = np.linspace(min(ll[:,1]),max(ll[:,1]),100)
        else:
            rankorderx = rankorderx_base[bs_samples]            
            rankordery = rankordery_base[bs_samples]
            #Add best fit line
            bscoeff = np.polyfit(rankorderx,rankordery,1)
            xrange_bs = np.linspace(min(rankorderx),max(rankorderx),100)            
        bestfitlines += [bscoeff]
    #Get upper and lower 95% percentile of y at each x
    yrangeLL = []
    yrangeUL = []
    for xi in xrange_bs:
        yrangeAll = [xi*bfl[0] + bfl[1] for bfl in bestfitlines]
        yrangePct = np.percentile(yrangeAll,(2.5, 97.5))
        yrangeLL += [yrangePct[0]]
        yrangeUL += [yrangePct[1]]
    ax.plot(xrange_bs,yrangeUL,'g:')
    ax.plot(xrange_bs,yrangeLL,'g:')
    
    #Get 95% CI
    ci_95_p = np.percentile(corr_bs_p,(2.5, 97.5))
    ci_95_s = np.percentile(corr_bs_s,(2.5, 97.5))
        
    print model_name
    print '\tPearson  r   = {:.3f} 95%CI[{:.3f},{:.3f}], p = {:.2e}'.format(
        corr_p[0],ci_95_p[0],ci_95_p[1],corr_p[1])
    #print '\tp = ' + str(corr[1])
    print '\tSpearman rho = {:.3f} 95%CI[{:.3f},{:.3f}], p = {:.2e}'.format(
        corr_s[0],ci_95_s[0],ci_95_s[1],corr_s[1])
    #Plot figure
    if pearson:
        ax.scatter(ll[:,1],ll[:,2])
        #Add best fit line
        coeff = np.polyfit(ll[:,1],ll[:,2],1)
        x = np.array([min(ll[:,1]),max(ll[:,1])])
        y = x*coeff[0] + coeff[1]
        #Find ideal position for text
        xrange = ax.get_xlim()
        textx = (xrange[1]-xrange[0])*.1+xrange[0]
        texty = .62
        #Specify axis label strings
        xlabel = 'negLL'
        ylabel = 'Participant p(error)'
    else:
        rankorderx = range(len(ll[:,1]))
        rankordery = ll[:,2].argsort()
        ax.scatter(rankorderx,rankordery)
        #Add best fit line
        coeff = np.polyfit(rankorderx,rankordery,1)
        x = np.array([min(rankorderx),max(rankorderx)])
        y = x*coeff[0] + coeff[1]
        #Find ideal position for text
        xrange = ax.get_xlim()
        yrange = ax.get_ylim()
        textx = (xrange[1]-xrange[0])*.1+xrange[0]
        texty = (yrange[1]-yrange[0])*.85+yrange[0]
        xlabel = 'negLL rank order'
        ylabel = 'Participant p(error) rank order '
        
    #Format presentation of correlation values
    if show_p:
        titlestr_p = 'r = {:.3}, p = {:.2e}'.format(corr_p[0],corr_p[1])
    else:
        titlestr_p = ''
    if show_s:        
        rho = r'$\rho$'
        titlestr_s = '{} = {:.3}, p = {:.2e}'.format(rho,corr_s[0],corr_s[1])
    else:
        titlestr_s = ''
    if show_p and show_s:
        titlestr_all = '{}\n{}'.format(titlestr_p,titlestr_s)
    else:
        if show_p:
            titlestr_all = '{}'.format(titlestr_p)
        elif show_s:
            titlestr_all = '{}'.format(titlestr_s)
        else:
            titlestr_all = ''
    ax.text(textx,texty,titlestr_all,fontsize=12)
    ax.plot(x,y,'--')
    ax.set_title(model_print)
    ax.set_xlabel(xlabel)
    #ax.set_title('{}\n{}'.format(titlestr_p, titlestr_s),fontsize=12)
    #ax.set_xlabel('negLL\n{}'.format(model_print))
    first_col = np.where(model_loc)    
    if first_col[1][0]==0:
        ax.set_ylabel(ylabel)
    else:
        ax.set_ylabel('')
        ax.set_yticklabels([])

        
# plt.savefig(savefilename)
# plt.cla()

#Also print the likelihood ratio test results for PACKER vs CopyTweak
# This is simply -2(LL_Packer - LL_CopyTweak)
#280618  - ok likelihood ratio test doesn't seem to make sense. Run bootstrap instead.
#Get global generation LL using these parameters
# global_ll_packer = trials.loglike(best_params['PACKER'],Packer,parmxform=False)
# global_ll_copytweak = trials.loglike(best_params['Copy and Tweak'],CopyTweak,parmxform=False)

# from scipy import stats as s2
# ll_ratio = -2.0 * (global_ll_packer - global_ll_copytweak)
# pval = s2.chi2.sf(ll_ratio,1);

# print "Likelihood ratio test:"
# print "\t chi_sq (1) = {}, p = {:.3}".format(ll_ratio,pval)




/Users/sourguyxian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


PACKER
	Pearson  r   = 0.681 95%CI[0.495,0.817], p = 1.30e-05
	Spearman rho = 0.662 95%CI[0.421,0.813], p = 2.73e-05
Copy and Tweak
	Pearson  r   = 0.093 95%CI[-0.328,0.440], p = 6.05e-01
	Spearman rho = 0.048 95%CI[-0.361,0.408], p = 7.91e-01
Hierarchical Sampling
	Pearson  r   = 0.099 95%CI[-0.329,0.455], p = 5.84e-01
	Spearman rho = -0.002 95%CI[-0.410,0.377], p = 9.92e-01
Hierarchical Sampling With Representativeness
	Pearson  r   = 0.547 95%CI[0.281,0.749], p = 9.81e-04
	Spearman rho = 0.558 95%CI[0.222,0.788], p = 7.47e-04


In [147]:
include


[26.0,
 121.0,
 112.0,
 95.0,
 32.0,
 89.0,
 85.0,
 38.0,
 6.0,
 114.0,
 98.0,
 5.0,
 72.0,
 94.0,
 17.0,
 99.0,
 54.0,
 80.0,
 51.0,
 117.0,
 18.0,
 88.0,
 40.0,
 15.0,
 25.0,
 66.0,
 41.0,
 111.0,
 115.0,
 67.0,
 75.0,
 53.0,
 44.0]

In [146]:
with open('pickles/ll150hi.p','wb') as f:
    pickle.dump(includes,f)


In [95]:
includes = {'includeCatAssign':includeCatAssign,'includeMatch':include,'includeMidBot':includeOld}

In [126]:
[info.participant[info.pptmatch==i].values[0] for i in include]

[243,
 121,
 160,
 143,
 194,
 89,
 85,
 38,
 6,
 114,
 98,
 5,
 72,
 94,
 17,
 99,
 54,
 80,
 51,
 117,
 122,
 222,
 141,
 15,
 25,
 66,
 150,
 159,
 247,
 67,
 75,
 239,
 131]

In [141]:
includeCatAssign = [info.participant[info.pptmatch==i].values[0] for i in include]
includeCatAssign

[243,
 121,
 160,
 143,
 194,
 89,
 85,
 38,
 6,
 114,
 98,
 5,
 72,
 94,
 17,
 99,
 54,
 80,
 51,
 117,
 122,
 222,
 141,
 15,
 25,
 66,
 150,
 159,
 247,
 67,
 75,
 239,
 131]

In [36]:
data_assign_file = 'experiment-midbot.db'
con = sqlite3.connect(data_assign_file)
info_e2 = pd.read_sql_query("SELECT * from participants", con)
con.close()

In [107]:
includeOld

[19,
 154,
 72,
 137,
 43,
 22,
 125,
 112,
 70,
 23,
 48,
 115,
 164,
 28,
 8,
 55,
 176,
 69,
 56,
 86,
 79,
 93,
 53,
 64,
 131,
 153,
 92,
 35,
 38,
 63,
 180,
 118,
 167]

In [47]:
funcs.getMatch('all','../cat-assign/data_utilities/cmp_midbot.db','Old')

array([[  0, 174],
       [  1, 114],
       [  2, 101],
       [  3,  80],
       [  4, 128],
       [  5, 115],
       [  6,  70],
       [  7,   7],
       [  8,  99],
       [  9,   3],
       [ 10,  44],
       [ 11, 122],
       [ 12,  47],
       [ 13,  66],
       [ 14, 170],
       [ 15,  64],
       [ 16, 120],
       [ 17,   8],
       [ 18,  79],
       [ 19,  60],
       [ 20, 150],
       [ 21,  15],
       [ 22,  58],
       [ 23,  17],
       [ 24, 103],
       [ 25, 131],
       [ 26,  19],
       [ 27,  74],
       [ 28, 102],
       [ 29,  67],
       [ 30, 159],
       [ 31, 147],
       [ 32,  43],
       [ 33,  98],
       [ 34,  41],
       [ 35,  36],
       [ 36,  68],
       [ 37, 119],
       [ 38, 112],
       [ 39, 136],
       [ 40,  53],
       [ 41,  92],
       [ 42,  62],
       [ 43, 168],
       [ 44, 167],
       [ 45,  78],
       [ 46,  83],
       [ 47, 178],
       [ 48, 145],
       [ 49,  91],
       [ 50,  50],
       [ 51,  56],
       [ 52,

In [102]:
info2 = info.copy()
stats2 = stats.loc[stats.participant.isin(includeOld)]
len(stats2)

33

In [103]:
stats

,area,between,bottom_only,bottom_used,correlation,drange,participant,top_and_bottom,top_only,top_used,within,xrange,xstd,yrange,ystd
0,3.999581,1.00000,0,1,0.000000,0.00,1,1,0,1,1.333333,2.00,1.000000,2.00,1.000000
1,0.062445,0.87500,0,1,0.000000,0.00,2,0,0,0,0.166667,0.25,0.125000,0.25,0.125000
2,0.000083,1.12500,0,0,0.000000,2.00,3,0,1,1,0.583333,2.00,0.790569,0.00,0.000000
3,0.062305,0.87500,0,1,-0.989949,-1.75,6,1,0,1,0.708333,0.25,0.125000,2.00,0.883883
4,0.000080,0.81250,1,1,0.000000,1.50,7,0,0,0,0.416667,1.50,0.572822,0.00,0.000000
5,0.312625,0.78125,0,0,0.377964,-0.25,8,0,0,1,0.479167,0.75,0.279508,1.00,0.369755
6,0.218712,0.75000,0,0,0.167444,-1.50,10,0,0,1,0.500000,0.25,0.108253,1.75,0.646505
7,4.000253,1.00000,0,1,0.000000,0.00,11,1,0,1,1.333333,2.00,1.000000,2.00,1.000000
8,1.968733,0.90625,0,1,0.432545,-0.25,13,1,0,1,1.104167,1.75,0.728869,2.00,0.941657
9,0.062450,0.87500,0,1,0.000000,0.00,15,0,0,0,0.166667,0.25,0.125000,0.25,0.125000


In [116]:
with open('pickles/midbot.p','rb') as f:
    trials = pickle.load(f)
trials.Set
# trials.task = 'generate'
# trials = Simulation.extractPptData(trials,includeOld)
# trials.Set

[{'categories': [array([30, 32, 48, 50]), array([], dtype=int64)],
  'participant': array([  1,   2,   6,   7,  13,  15,  19,  24,  27,  43,  44,  51,  55,
          62,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  76,  78,
          79,  80,  82,  89,  93,  97,  98, 100, 105, 122, 136, 137, 146,
         150, 151, 153, 154, 155, 157, 159, 160, 161, 162, 164, 165, 167,
         168, 170, 171, 172, 173, 174, 176, 178, 182]),
  'response': array([ 0,  8,  8,  8,  1,  8, 72, 80,  8, 75, 80, 20, 76, 72, 16, 27,  0,
          8, 48, 54,  0, 80, 73,  8, 80,  0, 78,  8, 75, 80, 48, 77, 72, 72,
          0, 72,  0, 45,  1, 80,  0, 39, 72,  5, 79, 80,  0, 43, 35, 61,  0,
         32, 80,  0,  0, 80,  8, 80, 76, 80, 71])},
 {'categories': [array([30, 32, 48, 50]), array([0])],
  'participant': array([  1,  67,  71,  78, 105, 136, 151, 160, 165, 170, 171]),
  'response': array([80,  9,  1, 36, 18,  8,  1, 72,  8, 37,  8])},
 {'categories': [array([30, 32, 48, 50]), array([ 0, 80])],
  'part

In [135]:
exclude

[12.0,
 19.0,
 9.0,
 91.0,
 23.0,
 4.0,
 93.0,
 28.0,
 35.0,
 43.0,
 36.0,
 87.0,
 118.0,
 100.0,
 113.0,
 68.0,
 92.0,
 37.0,
 16.0,
 65.0,
 60.0,
 2.0,
 24.0,
 8.0,
 49.0,
 108.0,
 7.0,
 56.0,
 101.0,
 45.0,
 20.0,
 61.0,
 11.0,
 79.0,
 73.0,
 3.0,
 81.0,
 29.0,
 52.0,
 34.0,
 109.0,
 110.0,
 120.0,
 90.0,
 31.0,
 1.0,
 13.0,
 21.0,
 42.0,
 96.0,
 59.0,
 102.0,
 97.0,
 83.0,
 105.0,
 48.0,
 14.0,
 71.0,
 119.0,
 86.0,
 64.0,
 57.0,
 46.0,
 62.0,
 22.0,
 39.0,
 0.0,
 106.0,
 78.0]

In [136]:
excludeCatAssign = [info.participant[info.pptmatch==i].values[0] for i in exclude]
excludeCatAssign

[12,
 163,
 9,
 91,
 204,
 127,
 93,
 251,
 213,
 43,
 36,
 87,
 118,
 187,
 124,
 68,
 92,
 37,
 16,
 65,
 135,
 145,
 24,
 200,
 49,
 108,
 7,
 129,
 101,
 45,
 20,
 61,
 11,
 79,
 73,
 169,
 81,
 29,
 52,
 139,
 207,
 110,
 120,
 90,
 31,
 132,
 13,
 21,
 42,
 96,
 59,
 102,
 97,
 83,
 241,
 238,
 126,
 71,
 119,
 240,
 64,
 57,
 46,
 62,
 22,
 39,
 0,
 106,
 78]

In [137]:
excludeMidBot = [funcs.getMatch(exclude[i],'../cat-assign/data_utilities/cmp_midbot.db','Old') for i in range(len(exclude))]
excludeMidBot
print(len(excludeMidBot))
print(len(exclude))
print(len(excludeCatAssign))

69
69
69


In [153]:
includes = {'includeCatAssign':excludeCatAssign,'includeMatch':exclude,'includeMidBot':excludeMidBot}
#Keep the names as include for easier extraction when fitting (include really indicates the participants included)

In [154]:
with open('pickles/ll150lo.p','wb') as f:
    pickle.dump(includes,f)

In [138]:
info

,categories,catflip,condition,counterbalance,lab,participant,pptmatch,stimuli,timetaken
0,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Middle,2,0,0,0,"[30, 32, 48, 50, 8, 80, 0, 72]",301471
1,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Bottom,4,0,5,5,"[12, 14, 30, 32, 80, 0, 72, 16]",335905
2,"[1, 1, 1, 1, 0, 0, 0, 0]",1,Middle,7,0,6,6,"[30, 32, 48, 50, 80, 54, 25, 0]",1187145
3,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Middle,0,0,7,7,"[30, 32, 48, 50, 7, 8, 2, 5]",196073
4,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Bottom,3,0,9,9,"[12, 14, 30, 32, 80, 72, 74, 78]",219109
5,"[1, 1, 1, 1, 0, 0, 0, 0]",1,Middle,1,0,11,11,"[30, 32, 48, 50, 77, 72, 74, 75]",221132
6,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Bottom,6,0,12,12,"[12, 14, 30, 32, 72, 80, 78, 79]",270451
7,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Middle,6,0,13,13,"[30, 32, 48, 50, 18, 27, 9, 54]",242215
8,"[0, 0, 0, 0, 1, 1, 1, 1]",0,Bottom,4,0,15,15,"[12, 14, 30, 32, 25, 32, 74, 73]",273268
9,"[1, 1, 1, 1, 0, 0, 0, 0]",1,Bottom,0,0,16,16,"[12, 14, 30, 32, 64, 73, 63, 72]",570489


In [167]:
a = np.array([[2,3],[4,2],[5,5]])
np.power(np.sum(np.power(a,2),axis=1),.5)

array([3.60555128, 4.47213595, 7.07106781])

In [173]:
a = 1.
a == 1

True